In [150]:
from navitia_client import Client
from  datetime import datetime
import requests
import pprint
import json
import pandas as pd
import numpy as np
from IPython.core.debugger import Tracer


In [72]:
df_stations = pd.read_csv('referentiel-gares-voyageurs.csv',sep=';')

In [108]:
df_stations[df_stations['Intitulé gare'].str.contains('(?i)Ferrand')]

,Code gare,Intitulé gare,Intitulé fronton de gare,Gare DRG,Gare étrangère,Agence gare,Région SNCF,Unité gare,UT,Nbre plateformes,...,Longitude WGS84,Latitude WGS84,Code UIC,TVS,Segment DRG,Niveau de service,SOP,RG,Date fin validité plateforme,WGS 84
1467,455,Clermont-Ferrand,Clermont-Ferrand,Oui,Non,AG Centre Est Rhône Alpin,REGION AUVERGNE,UG EEV AUVERGNE,NaN,1,...,3.100543,45.778945,87734004.0,CLF,a,2.0,NaN,NaN,NaN,"45.778945, 3.100543"
2626,627,Montferrand - Thoraise,Montferrand - Thoraise,Oui,Non,AG Centre Est Rhône Alpin,REGION BOURGOGNE FRANCHE-COMTE,UG Franche-Comté,NaN,1,...,5.907482,47.184741,87718106.0,MFH,c,1.0,NaN,NaN,NaN,"47.1847414, 5.9074815"


In [154]:
for y in df_stations.columns:
    if(df_stations[y].dtype == np.float64 or df_stations[y].dtype == np.int64):
          pass
    else:
          print df_stations[y].dtype

object
object
object
object
object
object
object
object
object
object
object
object
object
object
object


In [160]:
mask = np.column_stack([df_stations[col].str.contains("Clermont", na=False) for col in df_stations.columns if df_stations[col].dtype=='object' ])
df_stations.loc[mask.any(axis=1)]

,Code gare,Intitulé gare,Intitulé fronton de gare,Gare DRG,Gare étrangère,Agence gare,Région SNCF,Unité gare,UT,Nbre plateformes,...,Longitude WGS84,Latitude WGS84,Code UIC,TVS,Segment DRG,Niveau de service,SOP,RG,Date fin validité plateforme,WGS 84
994,64,Clermont la Pardieu,Clermont la Pardieu,Oui,Non,AG Centre Est Rhône Alpin,REGION AUVERGNE,UG EEV AUVERGNE,NaN,1,...,3.134163,45.766947,87782607.0,COP,b,1.0,NaN,NaN,NaN,"45.766947, 3.134163"
1162,1887,Clermont la Rotonde,Clermont la Rotonde,Oui,Non,AG Centre Est Rhône Alpin,REGION AUVERGNE,UG EEV AUVERGNE,NaN,1,...,3.090970,45.768095,87396895.0,RTD,c,1.0,NaN,NaN,NaN,"45.7680954, 3.0909704"
1467,455,Clermont-Ferrand,Clermont-Ferrand,Oui,Non,AG Centre Est Rhône Alpin,REGION AUVERGNE,UG EEV AUVERGNE,NaN,1,...,3.100543,45.778945,87734004.0,CLF,a,2.0,NaN,NaN,NaN,"45.778945, 3.100543"
1877,2298,Clermont de l'Oise,Clermont de l'Oise,Oui,Non,AG Manche-Nord,REGION DE PARIS-NORD,UG Picardie,NaN,1,...,2.417670,49.385310,87313304.0,CDO,b,2.0,NaN,NaN,NaN,"49.38531, 2.41767"
2209,2826,Clermont-en-Argonne,Clermont-en-Argonne,Oui,Non,AG Est-Européen,REGION LORRAINE,NaN,NaN,1,...,5.065060,49.106450,87175737.0,CTA,NaN,1.0,NaN,NaN,NaN,"49.10645, 5.06506"
2874,304,Monestier-de-Clermont,Monestier-de-Clermont,Oui,Non,AG Centre Est Rhône Alpin,REGION ALPES,UG EEV ALPES,NaN,1,...,5.633970,44.911996,87747592.0,MDC,c,1.0,NaN,NaN,NaN,"44.911996, 5.6339702"


In [161]:
df_stations.iloc[1467]

Code gare                                             455
Intitulé gare                            Clermont-Ferrand
Intitulé fronton de gare                 Clermont-Ferrand
Gare DRG                                              Oui
Gare étrangère                                        Non
Agence gare                     AG Centre Est Rhône Alpin
Région SNCF                               REGION AUVERGNE
Unité gare                                UG EEV AUVERGNE
UT                                                    NaN
Nbre plateformes                                        1
N° plateforme                                           1
Intitulé plateforme                      Clermont-Ferrand
Code postal                                         63000
Commune                                  Clermont-Ferrand
Département - Commune                              63-113
Département                                   Puy-de-Dôme
Région                               Auvergne-Rhône-Alpes
Longitude WGS8

In [111]:
def get_data(origin,destination,date):
    
    null='null'
    false='false'
    true='true'
    destination= "Brive-la-Gaillarde"
    destinationCode = "FRBVE"

    payload= {"origin":"Aurillac","originCode":"FRAUR","originLocation":null,"destination":"Clermont-Ferrand","destinationCode":destinationCode,"destinationLocation":null,"directTravel":false,"asymmetrical":false,"professional":false,"customerAccount":false,"oneWayTravel":true,"departureDate":"2017-08-21T13:00:00","returnDate":null,"travelClass":"SECOND","country":"FR","language":"fr","busBestPriceOperator":null,"busOnly":false,"passengers":[{"travelerId":null,"profile":"ADULT","age":26,"birthDate":null,"fidelityCardType":"NONE","fidelityCardNumber":null,"commercialCardNumber":null,"commercialCardType":"NONE","promoCode":"","lastName":null,"firstName":null,"phoneNumer":null,"hanInformation":null}],"animals":[],"bike":"NONE","withRecliningSeat":false,"physicalSpace":null,"fares":[],"withBestPrices":false,"highlightedTravel":null,"nextOrPrevious":false,"source":"FORM_SUBMIT","targetPrice":null,"han":false,"$initial":true,"$queryId":"SRlEN"}

    payload = json.dumps(payload)
    payload = payload.replace("\"null\"","null").replace("\"false\"","false").replace("\"true\"","true")
    return payload

In [112]:
data = get_data('Angouleme','Tours',datetime(2017,8,21,8,0,0))

In [113]:
print data

{"origin": "Aurillac", "originLocation": null, "nextOrPrevious": false, "destinationCode": "FRBVE", "$queryId": "SRlEN", "busOnly": false, "customerAccount": false, "$initial": true, "asymmetrical": false, "departureDate": "2017-08-21T13:00:00", "withBestPrices": false, "directTravel": false, "fares": [], "targetPrice": null, "passengers": [{"travelerId": null, "profile": "ADULT", "phoneNumer": null, "fidelityCardType": "NONE", "birthDate": null, "fidelityCardNumber": null, "commercialCardType": "NONE", "firstName": null, "commercialCardNumber": null, "lastName": null, "age": 26, "promoCode": "", "hanInformation": null}], "animals": [], "han": false, "language": "fr", "source": "FORM_SUBMIT", "country": "FR", "destination": "Clermont-Ferrand", "returnDate": null, "destinationLocation": null, "busBestPriceOperator": null, "oneWayTravel": true, "bike": "NONE", "highlightedTravel": null, "withRecliningSeat": false, "professional": false, "physicalSpace": null, "travelClass": "SECOND", "or

In [117]:
response = requests.request("POST", URL, data=data, headers=headers)

In [118]:
print type(response.json())
result_dict = response.json()['results']

<type 'dict'>


In [119]:
print len(result_dict)


3


In [120]:
result_dict[0].keys()

[u'origin',
 u'pushProposal',
 u'priceProposals',
 u'hasPerturbations',
 u'destinationCode',
 u'transporters',
 u'segments',
 u'destination',
 u'pushPrice',
 u'containsTgvPopFares',
 u'arrivalDate',
 u'journeyId',
 u'unsellableReason',
 u'departureDate',
 u'pushProposalType',
 u'duration',
 u'realTimeInformations',
 u'transactionId',
 u'id',
 u'originCode']

In [121]:
pd.DataFrame.from_records(result_dict[0]['segments'])

,arrivalDate,availablePhysicalSpaces,baseClasses,bookingType,boundaryPoint,carrierCode,departureDate,destination,destinationCode,destinationDetail,...,showRating,tctCodeList,ticketProvider,trainEquipment,trainNumber,trainPeriod,trainType,transporter,transporterRating,vehicleType
0,2017-08-21T14:03:00,[B],None,RI,None,SN,2017-08-21T12:55:00,FIGEAC GARE,FRFIG,"{u'cityCode': u'FRFHV', u'cityLabel': u'FIGEAC...",...,False,[],RR,None,71209,BLUE,TER,TER,None,TRAIN
1,2017-08-21T16:43:00,[B],None,RI,None,SN,2017-08-21T15:30:00,BRIVE LA GAILLARDE,FRBVE,"{u'cityCode': None, u'cityLabel': None, u'outs...",...,False,[],RR,None,70258,BLUE,TER,TER,None,TRAIN


In [122]:
result_dict[0]['priceProposals'].keys()

[u'FLEX']

In [123]:
pd.DataFrame.from_dict(result_dict[0]['priceProposals'])

,FLEX
amount,25.7
amountWithMandatorySpaceComfort,25.7
animalDetails,[]
appliedDiscount,0
bookingFee,None
containsTgvPopFare,False
currency,EUR
detailMessages,[]
eligibleForCancellationInsurance,False
flags,[]


In [124]:
pd.DataFrame.from_dict(result_dict[0]['transporters'])

,0
0,TER
1,TER


In [12]:
result_dict[0]['unsellableReason']

u'NONE'

In [125]:
def convert_result_to_dataframe(result):
    
    df =pd.DataFrame()
    #Tracer()() #this one triggers the debugger
 
    df = pd.DataFrame.from_records(result['segments'])
    df[['departureDate','arrivalDate']] = df[['departureDate','arrivalDate']].applymap(lambda x : pd.to_datetime(x, format ="%Y-%m-%dT%H:%M:%S"))
    df['total_duration'] = df['arrivalDate'].iloc[-1]-df['departureDate'].iloc[0]
    df['trip_name']=u' to '.join([result['origin'],result['destination']])
    df['trip_id']=result['id']
    df['unsellableReason']=result['unsellableReason']
    df['TGV']='TGV' in df['transporter'].values
    df['total_trains']= df.shape[0]

    
    for price_key in result['priceProposals'].keys():
        df[''.join(["price_",price_key])]=result['priceProposals'][price_key]['amount']
                   
    return df

def concat_results(json_result):
    df =pd.DataFrame()
    for result in json_result:
        
        try:
            df_result = convert_result_to_dataframe(result)
            df = pd.concat([df,df_result],axis =0)
        except Exception as e:
            print "error ....................."
    return df
    
    
    

In [126]:
len(result_dict)

3

In [127]:
type(result_dict)

list

In [148]:
test_df.groupby('trip_id')[['duration']].mean()

,duration
trip_id,
6b11b79b-e02f-45c2-8c9a-d11373575ec9,6540000
823b67bc-9eb8-4fd5-b5ea-f6aa8c7d8b3a,4230000
a79bb39c-9cfd-4b47-8e64-8864c3a4b3bf,6480000


In [129]:
test_df.iloc[1].arrivalDate-test_df.iloc[0].departureDate

Timedelta('0 days 03:48:00')